In [10]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from torch.utils import data

# Cargar YOLOv8
model = YOLO('yolov8n.pt')  # Carga el modelo YOLOv8n preentrenado

# Obtener la lista de archivos en la carpeta
folder_path = "images"  # Carpeta que contiene las imágenes
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', 'png'))]  # Lista de archivos de imagen

# Inicializar una lista para almacenar todas las imágenes con objetos detectados
combined_image = None  # Imagen combinada inicializada como None

# Iterar sobre las imágenes y detectar objetos
for image_file in image_files:
    # Cargar imagen
    image_path = os.path.join(folder_path, image_file)  # Ruta completa de la imagen
    image = cv2.imread(image_path)  # Cargar la imagen usando OpenCV
    if image is None:
        continue  # Saltar archivos que no se puedan cargar como imágenes
    height, width, channels = image.shape  # Obtener dimensiones de la imagen

    # Detectar objetos
    results = model(image)  # Ejecutar el modelo YOLO para detectar objetos en la imagen

    # Dibujar las cajas alrededor de los objetos detectados
    for result in results:
        boxes = result.boxes.xyxy.numpy()  # Coordenadas de las cajas (esquinas superior izquierda e inferior derecha)
        confidences = result.boxes.conf.numpy()  # Confianza de cada detección
        class_ids = result.boxes.cls.numpy().astype(int)  # IDs de clase de los objetos detectados
        labels = [model.names[int(cls)] for cls in class_ids]  # Nombres de las clases detectadas

        for box, confidence, label in zip(boxes, confidences, labels):
            if confidence > 0.1:  # Solo dibujar si la confianza es mayor que 0.1
                x1, y1, x2, y2 = map(int, box)  # Coordenadas de la caja convertidas a enteros
                color = (0, 255, 0)  # Color verde para las cajas
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)  # Dibujar la caja
                fontScale = 1 # Ajusta este valor para aumentar el tamaño del texto
                text = f"{label} {confidence:.2f}"  # Agregar la confianza al texto
                cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, fontScale, color, 2)  # Dibujar el texto

    # Redimensionar la imagen para que todas tengan la misma altura
    desired_height = 600  # Especifica la altura deseada para todas las imágenes
    ratio = desired_height / height  # Calcular el ratio de redimensionamiento
    image_resized = cv2.resize(image, (int(width * ratio), desired_height))  # Redimensionar la imagen

    # Concatenar la imagen actual con las imágenes anteriores
    if combined_image is None:
        combined_image = image_resized  # Si es la primera imagen, establecer como imagen combinada
    else:
        combined_image = np.hstack((combined_image, image_resized))  # Concatenar horizontalmente

# Mostrar la imagen resultante con todas las imágenes combinadas
cv2.imshow("Combined Image", combined_image)  # Mostrar la imagen combinada
cv2.waitKey(0)  # Esperar a que se presione una tecla
cv2.destroyAllWindows()  # Cerrar todas las ventanas de OpenCV



0: 448x640 3 bananas, 12 apples, 6 oranges, 272.9ms
Speed: 6.2ms preprocess, 272.9ms inference, 3.1ms postprocess per image at shape (1, 3, 448, 640)
